In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 

In [ ]:
traindf = pd.read_hdf("../data/traindf_clean.hdf")
testdf = pd.read_hdf("../data/test_clean.hdf")

In [ ]:
# Toch niet met priceband want categorie
testdf = testdf.drop("PriceBand", axis=1)
traindf = traindf.drop("PriceBand", axis=1)

In [ ]:
print(list(traindf.columns))
print(list(testdf.columns))

In [ ]:
# split label and other variables
x_train, y_train = traindf, traindf["position"]
positions = pd.DataFrame(y_train).set_index(traindf.srch_id)

# 1 procent van totale data
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.99, random_state=42, shuffle=False)
x_train.drop(columns=["position", "importance", "click_bool", "booking_bool"])

# x_test, y_test = testdf.drop(columns=["position", "click_bool", "booking_bool"]), testdf["position"]
x_test = testdf

x_train.shape, y_train.shape, x_test.shape

In [ ]:
params = {'objective': 'rank:pairwise', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6,  'n_estimators': 1000}

# TODO: ik snap niet precies wat de groups zijn in XGBRanker
query_lengths = []
for group_unique in x_train.groupby('srch_id'):
    query_lengths.append(len(group_unique[1]))
    

In [ ]:
model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, query_lengths, verbose=True)

In [ ]:
pred = model.predict(x_train)

predictions_sorted = []
x_train["position"] = y_train

# Sort predictions for each group SEPERATELY
for srchid, group in x_train.groupby('srch_id'):
    
    pred = model.predict(group.drop("position", axis=1))

    predictions = [x for _,x in sorted(zip(pred, group.position), reverse=True)] # TODO: Reverse = False toch? Hoezo werkt dit..
    predictions_sorted.append(predictions)
    
predictions_sorted = [item for sublist in predictions_sorted for item in sublist]

In [ ]:

x_traincopy["predicted_rank"] = predictions_sorted

In [ ]:
x_traincopy.to_pickle("../pickles/XGBoost_solution.pkl")

## Test set only

In [ ]:
pred = model.predict(x_test)

x_test["predicted_rank"] = pred
predictions_sorted = []

# Sort predictions for each group SEPERATELY
for srchid, group in x_test.groupby('srch_id'):
    
    predictions = [x for _,x in sorted(zip(group.predicted_rank, group.prop_id))]
    predictions_sorted.append(predictions)
    
predictions_sorted = [item for sublist in predictions_sorted for item in sublist]

In [ ]:
# final_df = x_test[["srch_id", "prop_id"]]
# final_df.to_csv("submission5mei.csv", columns=["srch_id", "prop_id"], index=False)

In [ ]:
# # Test if it worked
# test = pd.read_csv("submission5mei.csv")
# test.head()

In [ ]:
importances = pd.DataFrame(x_test.columns, model.feature_importances_)
importances.head(10)
